In [1]:
import numpy as np
import matplotlib.pyplot as plt
import arviz as az

from matplotlib.pyplot import cm
from scipy.signal import find_peaks

In [2]:
# Set the style of the plots to a more beautiful format
az.style.use('arviz-darkgrid')

In [3]:
counter=1
#EEE = []
for nu in [0.0305,0.0301,0.03,0.02998,0.029971,0.0299695]:
    
    L = 2 * np.pi
    nx = 8192

    t0 = 0 
    tN = 8
    dt = 0.0005
    nt = int((tN - t0) / dt)

    # wave number mesh
    k = np.arange(-nx/2, nx/2, 1)

    t = np.linspace(start=t0, stop=tN, num=nt)
    x = np.linspace(start=0, stop=L, num=nx)

    # solution mesh in real space
    u = np.ones((nx, nt))
    # solution mesh in Fourier space
    u_hat = np.ones((nx, nt), dtype=complex)

    u_hat2 = np.ones((nx, nt), dtype=complex)

    # initial condition 
    #u0 = np.cos((2 * np.pi * x) / L) + 0.1 * np.cos((4 * np.pi * x) / L)
    u0 = - np.sin(x)

    # Fourier transform of initial condition
    u0_hat = (1 / nx) * np.fft.fftshift(np.fft.fft(u0))

    u0_hat2 = (1 / nx) * np.fft.fftshift(np.fft.fft(u0**2))

    # set initial condition in real and Fourier mesh
    u[:,0] = u0
    u_hat[:,0] = u0_hat

    u_hat2[:,0] = u0_hat2

    # Fourier Transform of the linear operator
    FL = (((2 * np.pi) / L) * k) ** 2 - nu * (((2 * np.pi) / L) * k) ** 4
    # Fourier Transform of the non-linear operator
    FN = - (1 / 2) * ((1j) * ((2 * np.pi) / L) * k)

    # resolve EDP in Fourier space
    for j in range(0,nt-1):
        uhat_current = u_hat[:,j]
        uhat_current2 = u_hat2[:,j]
        if j == 0:
            uhat_last = u_hat[:,0]
            uhat_last2 = u_hat2[:,0]
        else:
            uhat_last = u_hat[:,j-1]
            uhat_last2 = u_hat2[:,j-1]

        # compute solution in Fourier space through a finite difference method
        # Cranck-Nicholson + Adam 
        u_hat[:,j+1] = (1 / (1 - (dt / 2) * FL)) * ( (1 + (dt / 2) * FL) * uhat_current + ( ((3 / 2) * FN) * (uhat_current2) - ((1 / 2) * FN) * (uhat_last2) ) * dt )
        # go back in real space
        u[:,j+1] = np.real(nx * np.fft.ifft(np.fft.ifftshift(u_hat[:,j+1])))
        u_hat2[:,j+1] = (1 / nx) * np.fft.fftshift(np.fft.fft(u[:,j+1]**2))


    # plot the result
    fig = plt.figure(figsize=(20,10))

    ax = fig.add_subplot(1, 2, 1)

    xx, tt = np.meshgrid(x, t)
    levels = np.arange(-3, 3, 0.01)
    cs = ax.contourf(xx, tt, u.T, cmap=cm.jet)
    #ax.colorbar(cs)

    ax.set_xlabel("x")
    ax.set_ylabel("t")
    #ax.set_title(f'L = {np.round(L,4)} ;' +r'$\nu$' + f'={nu}')

    # Second subplot
    ax = fig.add_subplot(1, 2, 2, projection='3d')

    ax.plot_surface(xx, tt, u.T,cmap='viridis', edgecolor='none')
    ax.set_xlabel('x')
    ax.set_ylabel('t')
    ax.set_zlabel('u');
    ax.set_title(r'$\nu$'+f'={nu}')


    plt.savefig(f'KS3Dplots_{counter}.png')
    print(counter)
    counter=counter+1
plt.show()


1
2
3
